In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_name_or_path = "openai/whisper-large-v3"
torch.cuda.empty_cache()
print(device, torch_dtype)

cuda:0 torch.float16


In [3]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_name_or_path,
    #torch_dtype=torch_dtype,  # Use float16 precision
    low_cpu_mem_usage=True,  # Optimize memory during loading
    use_safetensors=True
)
model.to(device)

# Load processor (unchanged)
processor = AutoProcessor.from_pretrained(model_name_or_path)

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 7.74 GiB of which 50.31 MiB is free. Process 321697 has 6.42 GiB memory in use. Including non-PyTorch memory, this process has 718.00 MiB memory in use. Of the allocated memory 592.39 MiB is allocated by PyTorch, and 33.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
generate_kwargs = {
    "max_new_tokens": 444,
    "num_beams": 1,
    "condition_on_prev_tokens": False,
    # zlib compression ratio threshold (in token space)
    "compression_ratio_threshold": 1.35,
    "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
    "logprob_threshold": -1.0,
    "no_speech_threshold": 0.6,
    "return_timestamps": True,
    "max_new_tokens":128,
    "chunk_length":15,
    "batch_size":16,
    "feature_extractor": processor.feature_extractor,
    "tokenizer": processor.tokenizer,
    "device":device,
    "torch_dtype":torch_dtype,
    "language": "<|en|>",
    "task": "transcribe"
}